In [ ]:
<h4>RNN</h4>
<img src="./rnn.png" alt="rnn" />
<a href="https://en.wikipedia.org/wiki/Recurrent_neural_network#">Source</a> 
<p>The RNN uses a feedback loop to store a previous state. This memory can be unrolled into a previous state, current state and next state. RNNs suffer from a vanishing gradient problem</p>
<p>Despite the VG problem, the traditional RNN architectures are a basis for solving sequence type tasks</p>



In [73]:
#https://towardsdatascience.com/pytorch-basics-how-to-train-your-neural-net-intro-to-rnn-cb6ebc594677
import math

seq = list(range(20))
print(len(seq))
num = math.floor(len(seq)/3)
seq[0:num]
seqlen=6
batches = [seq[x*seqlen:x*seqlen+num] for x in range(3)]
print(batches)

20
[[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17]]


In [76]:
class MakeBatch:
    def __init__(self,batch_size=3, seqlen=6):
        self.batch_size=batch_size
        self.seqlen = seqlen
        self.startValue = 0
    def next(self):
        data = range(self.startValue , self.startValue + self.seqlen)
        self.startValue += self.seqlen
        return list(data)

g = MakeBatch()
print(g.next())
print(g.next())

[0, 1, 2, 3, 4, 5]
[6, 7, 8, 9, 10, 11]


In [72]:
#simple python generator example
class GenExample:
    def __init__(self,seqlen):
        self.seqlen=seqlen
        self.i = 0
    def get_gen(self):
        while self.i < (self.seqlen):
            yield self.i
            self.i+=1
        
        
# lessons 1) cant use a for loop
# 2) return a generator object, not the data
# 3) the client API is next(gen object)
# data pipelines in cloud you have to build APIs as generator objects
# cascade gen objects genobj2(genobj1)
y = GenExample(5)
z = y.get_gen()
print(next(z))
print(next(z))
print(next(z))
print(next(z))


0
1
2
3


In [88]:
class GeneratorBatch:
    def __init__(self):
        self.seqlen=6
        self.batch=3
        self.idx =0
        self.num_batch=0
        self.start = 0
    def get_gen(self):
        while self.num_batch < self.batch:
            yield list(range(self.start, self.start+self.seqlen))
            self.num_batch += 1
            self.start += self.seqlen
            
g = GeneratorBatch()
y = g.get_gen()
try:
    print(next(y))
    print(next(y))
    print(next(y))
    print(next(y))
except StopIteration:
    pass 



[0, 1, 2, 3, 4, 5]
[6, 7, 8, 9, 10, 11]
[12, 13, 14, 15, 16, 17]


In [102]:
import time
import random
import unidecode
import string
import re

import matplotlib.pyplot as plt
import torch

torch.backends.cudnn.deterministic = True
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

DEVICE = torch.device('cpu')

TEXT_PORTION_SIZE = 200

NUM_ITER = 5000
LEARNING_RATE = 0.005
EMBEDDING_DIM = 100
HIDDEN_DIM = 128

print('Device:', DEVICE)

Device: cpu


In [106]:
with open('./covid.txt') as fh:
    lines = fh.read()
#print(lines)
remove_chars = unidecode.unidecode(lines)
remove_chars = re.sub(" +"," ", remove_chars)
TEXT_LENGTH = len(remove_chars)
print(len(remove_chars))

84657


In [107]:
random.seed(RANDOM_SEED)
def random_portion(textfile):
    start_index = random.randint(0, TEXT_LENGTH-TEXT_PORTION_SIZE)
    end_index = start_index + TEXT_PORTION_SIZE + 1
    return textfile[start_index:end_index]

print(random_portion(remove_chars))

heck with the vaccine provider.

If you're traveling away from Madison before your second dose, we encourage you to get your first dose on campus now and obtain a second dose once you reach your new lo


In [111]:
def char_to_tensor(text):
    lst = [string.printable.index(c) for c in text]
    tensor = torch.tensor(lst).long()
    return tensor
print(char_to_tensor("abcDEF"))


tensor([10, 11, 12, 39, 40, 41])


In [113]:
def draw_random_sample(text_file):
    text_long = char_to_tensor(random_portion(text_file))
    inputs = text_long[:-1]
    targets = text_long[1:]
    return inputs, targets
draw_random_sample(remove_chars)

(tensor([ 0, 94, 24, 27, 94, 21, 10, 29, 14, 27, 94, 24, 15, 94, 26, 30, 10, 27,
         10, 23, 29, 18, 23, 14, 94, 10, 23, 13, 94, 34, 24, 30, 27, 94, 37, 10,
         13, 16, 14, 27, 94, 37, 10, 13, 16, 14, 94, 18, 23, 13, 18, 12, 10, 29,
         14, 28, 94, 63, 11, 30, 18, 21, 13, 18, 23, 16, 94, 10, 12, 12, 14, 28,
         28, 94, 16, 27, 10, 23, 29, 14, 13, 75, 63, 96, 54, 14, 14, 94, 29, 17,
         14, 94, 56, 23, 18, 31, 14, 27, 28, 18, 29, 34, 94, 43, 14, 10, 21, 29,
         17, 94, 54, 14, 27, 31, 18, 12, 14, 28, 94, 32, 14, 11, 28, 18, 29, 14,
         94, 15, 24, 27, 94, 22, 24, 27, 14, 94, 18, 23, 15, 24, 27, 22, 10, 29,
         18, 24, 23, 94, 10, 11, 24, 30, 29, 94, 26, 30, 10, 27, 10, 23, 29, 18,
         23, 14, 94, 10, 23, 13, 94, 18, 28, 24, 21, 10, 29, 18, 24, 23, 75, 96,
         96, 58, 17, 18, 21, 14, 94,  1,  4, 94, 13, 10, 34, 28, 94, 18, 28, 94,
         29, 17]),
 tensor([94, 24, 27, 94, 21, 10, 29, 14, 27, 94, 24, 15, 94, 26, 30, 10, 27, 10,
         

In [114]:
#MODEL single row RNN
class RNN(torch.nn.Module):
    def __init__(self,input_size, hidden_size, embed_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.embed = torch.nn.Embedding(num_embeddings=input_size, embedding_dim=embed_size)
        self.rnn = torch.nn.LSTMCell(input_size=embed_size, hidden_size=hidden_size)
        self.fc = torch.nn.Linear(hidden_size, output_size)
    def forward(self, character, hidden, cell_state):
        #char=[batch_size, 1]
        #batch_size, embedding = 1, embedding
        embedded = self.embed(character)
        (hidden, cell_state) = self.rnn(embedded, (hidden, cell_state))
        # output: [batch_size, output_size] = [1, output]
        # hidden: [batch_size, hidden_dim] = [1, hidden_dim]
        # cell: [batch_size, hidden] = [1, hidden_dim]
        output = self.fc(hidden)
        return output, hidden, cell_state
    def init_zero_state(self):
        init_hidden = torch.zeros(1, self.hidden_size).to(DEVICE)
        init_cell = torch.zeros(1, self.hidden_size).to(DEVICE)
        return (init_hidden, init_cell)

torch.manual_seed(RANDOM_SEED)
model = RNN(len(string.printable), EMBEDDING_DIM, HIDDEN_DIM,len(string.printable))
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [ ]:
# TRAINING
def evaluate(model, prime_str='A', predict_len=100,temperature=100):
    ## based on https://github.com/spro/practical-pytorch/
    ## blob/master/char-rnn-generation/char-rnn-generation.ipynb
    (hidden, cell_state) = model.init_zero_state()
    prime_input = 
    predicted = prime_str

    for p in range(len(prime_str) - 1):
        inp = prime_input()
        _, hidden, cell_state = model()
    inp = prime_input[-1].unsqueeze(0)